In [43]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score, LeaveOneOut, GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model, svm

from itertools import combinations
from textwrap import wrap

# Exploring covariance between properties across scales
**Authors**: Benjamin Jasperson, Harley T. Johnson

**GitHub URL**: https://github.com/bjasperson/property-covariance-demo

# Background Information

why
compromise on modeling: accuracy/first-principles vs. speed and domain size.

what

goals

We're interested in uncovering relationships between fundamental microscopic properties and GB energy.
Desire a scalar value to compare against.
Derived a scaling coefficient.
For purposes here, you don't need to worry about how this coefficient is defined (see the paper for details).

# Download Data
To begin, we need to import our project data.
Collecting this data ....

The data we will use is originally from [this manuscript](https://arxiv.org/abs/2411.16770), [published here](https://doi.org/10.1016/j.actamat.2025.120722). You can find the original paper repo [here](https://github.com/Johnson-Research-Group/gb_covariance).

**Task**: Using pandas, download the CSV file with this [link](https://github.com/bjasperson/property-covariance-demo/blob/main/data/gb_data.csv?raw=true) and look at the first five rows.

In [ ]:
url_link = "https://github.com/bjasperson/property-covariance-demo/blob/main/data/gb_data.csv?raw=true"
df_data = pd.read_csv(url_link, index_col=0)
df_data.head()

# Data review
It is important to get a feel for the data that you are working with. 

**Task**: import the label csv file, located at `./data/label_dict.csv`. Convert it to a dictionary for use with plotting.

In [45]:
def import_label_dict(label_dict_path = "./data/label_dict.csv"):
    """import the ./data/label_dict.csv file, convert it to a dictionary

    Args:
        label_dict_path (str, optional): path to csv file with key/value pairs. Defaults to "./data/label_dict.csv".

    Returns:
        dict: a dictionary with df column names as keys, and human-readable label as value
    """
    
    # useful for plotting purposes
    df_label_dict = pd.read_csv(label_dict_path)
    label_dict = df_label_dict.to_dict(orient="records")[0]
    return label_dict

We now want to look at how the different fundamental properties relate with each other and our Quantity of Interest (GB energy coefficient). To do this, we need to create a plotting function.

**Task**: create a function to plot a set of parameters from the dataframe using Seaborn pairplot. Explore different combinations of pairplots for the data.

In [ ]:
def pairplot_fun(df, 
                 params_list,  
                 label_dict, 
                 height=1.5,
                 #xlims = False, 
                 ):
    """create pairplot for select indicator properties

    Args:
        df (pandas.DataFrame): dataframe of data that was previously imported
        params_list (list): list of parameter strings to use
        label_dict (dict): the label dict from above, to convert label keys to human readable format
        height (float): pairplot height. Defaults to 2.0.
        xlims (list): list of limits to use for plotting
    """
    params_list.extend(['species'])
    X = df[params_list]
    X.columns = [label_dict[x] for x in X.columns.to_list()]
    sns.set_theme(style="white", font_scale = 1)
    marker_list = ['o','^','v','<','>','s','D','p','X','*','.','P']
    g = sns.pairplot(X, hue='species', corner=True, markers = marker_list[0:len(df.species.drop_duplicates())],
                     plot_kws=dict(s=50, linewidth=0.1, rasterized = True), 
                     height=height,
                     diag_kind='hist')
    sns.move_legend(g, "upper right", bbox_to_anchor = (0.85,1))

    # if xlims != False:
    #     for i in range(len(g.axes)):
    #         for j in range(len(g.axes[i])):
    #             g.axes[i][j].set_xlim(xlims)
    #             g.axes[i][j].set_ylim(xlims)


label_dict = import_label_dict()
plot_param_list = ["lattice_constant_fcc", "lattice_constant_bcc", 'intr_stack_fault_energy_fcc', 'unstable_stack_energy_fcc']
pairplot_fun(df_data, plot_param_list, label_dict)

# Identifying indicator properties

**Task**: run K-fold cross-validation. 

# Develop regression model
Linear regression versus SVR.

**Task**: Write a function that returns a linear regression pipeline. It should include:
- StandardScaler: for scaling the input values; helpful for SVR, used in LR for consistency. 
- KNNImputer: for filling in the missing property values in our data. Set `n_neighbors=2` and `keep_empty_features=True`
- LinearRegression: the linear regression model.
Combine these in a `Pipeline` to facilitate easy use .

In [26]:
def linear_regr_pipeline():
    """creates a linear regression pipeline for modeling purposes

    Returns:
        sklearn.pipeline.Pipeline: linear regression pipeline
    """

    # Linear regression selection pipeline, used in manuscript
    imput = KNNImputer(n_neighbors=2,
                       keep_empty_features=True)
    model = linear_model.LinearRegression()

    pipe = Pipeline(steps=[('scale', StandardScaler()),
                            ('imp', imput),
                            ('lr', model)])
    return pipe

pipe = linear_regr_pipeline()

Now, we need to make a list of the properties we are interested in considering as indicator properties for our model. A starting point is provided below.

**If time**: explore different properties, and see if any new additions result in improved models!

In [27]:
# list of parameters to explore
params_list = ['lattice_constant_fcc',
                'bulk_modulus_fcc', 'c11_fcc', 'c12_fcc', 'c44_fcc',
                'extr_stack_fault_energy_fcc', 'intr_stack_fault_energy_fcc', 'unstable_stack_energy_fcc'
                ]

Using the parameter list, we want to make a list of possible combinations up to a certain number of factors. 

**Task**: write a function that uses the `combinations` package to make a list of lists, with possible combinations of factors up to `n_factor_max`.

In [ ]:
def create_subsets_list(factor_list, n_factor_max):
    subsets = []
    for n in range(1, (n_factor_max+1)):
        for subset in combinations(factor_list, n):
            subsets.append(list(subset))
    return subsets

subsets = create_subsets_list(params_list, n_factor_max = 2)
print(subsets)

We now have our list of possible factor combinations to consider. If we fit our model to the entire dataset, we aren't getting a realistic picture of the test error. So, we will use the `cross_val_score` package from scikit-learn to perform cross validation. Read more about it [here](https://scikit-learn.org/stable/modules/cross_validation.html).

**Task**: write a function that iterates over each subset of the possible factor combinations. For each subset, use the `cross_val_score` package to perform cross-validation and return a score. 

In [29]:
def factor_select_cv(df, factors, pipe, subsets, label="coeff", cv=5, scoring='r2'):
    """evaluates combinations of factors
    
    return a dataframe that includes one row per factor combination. list of parameters w/ cv score
    """
    X = df[factors]
    y = df[label]
    
    cv_score_mean = []
    cv_score_std = []
    num_factors = []
    for i, subset in enumerate(subsets):
        print(f"{i} of {len(subsets)}")
        print('current subset: ', subset)
        score = cross_val_score(pipe, X[subset], y, cv=cv, scoring=scoring, n_jobs = -1)
        print('score mean: ', np.mean(score))
        cv_score_mean.append(np.mean(score))
        cv_score_std.append(np.std(score))
        num_factors.append(len(subset))

    df_results = pd.DataFrame({'factors': subsets,
                               'num_factors': num_factors,
                               'cv_score': cv_score_mean,
                               'cv_score_std': cv_score_std})
    df_results = df_results.sort_values('cv_score', ascending=False)

    return df_results

In [ ]:
# pipe = TransformedTargetRegressor(regressor=pipe,
#                                   transformer=StandardScaler())
filename = "kfold_models_lr"

cv = RepeatedKFold(n_splits=10, n_repeats=5)
df_results = factor_select_cv(df_data,
                              params_list,
                              pipe, 
                              subsets, 
                              cv=cv, 
                              scoring='neg_root_mean_squared_error')

**If time**: repeat the steps above using different modeling approaches, e.g., support vector regression. Compare performance of the models, and try to understand why some perform better than others.

In [ ]:
df_results.sort_values("cv_score", ascending=False)

Pick the top performing model and make a regression model to use. Make predictions and compare predicted versus actual.

In [32]:
def add_pred(df_in, pipe_in, factors, label="coeff"):
    X = df_in[factors]
    y = df_in[label]
    pipe_in.fit(X, y)
    y_pred = pipe_in.predict(X)
    return y_pred

df_data["coeff_pred"] = add_pred(df_data, pipe, ["c12_fcc", "c44_fcc"])

In [ ]:
df_data.head()

**Task**: write a function that takes in the dataframe, plots the actual coefficient on the horizontal axis, and plots the predicted coefficient on the vertical axis. Use seaborn scatterplot with "species" as the hue. Set the aspect ratio to "equal" for each axis, and plot a line along the diagonal to represent a perfect prediction. 

In [34]:
def pred_vs_actual_plot(df, 
                        figsize = (5,5),
                        ):
    sns.set_style("whitegrid")
    fig,ax = plt.subplots(figsize = (figsize[0],figsize[1]))
    sns.scatterplot(data=df, x='coeff', y='coeff_pred',hue='species',style = 'species')
    ax.plot(np.linspace(0,max(df['coeff']),50),
           np.linspace(0,max(df['coeff']),50))
    ax.set_xlabel(r"actual coefficient [$J/m^2$]", fontsize=8)
    ax.set_ylabel(r"predicted coefficient [$J/m^2$]", fontsize=8)
    ax.tick_params(labelsize=8)      
    ax.axes.set_aspect('equal')    

In [ ]:
pred_vs_actual_plot(df_data)

# Make predictions from first-principles indicator properties

Now, we will import the DFT data that we will use with our model to make inferred predictions based on the indicator properties.

In [ ]:
url_link = "https://github.com/bjasperson/property-covariance-demo/blob/main/data/gb_dft.csv?raw=true"
df_dft = pd.read_csv(url_link, index_col=0)
df_dft.head()

Filter your model performance list from above based on the DFT properties available, and select the top performing model.

In [ ]:
def filter_model_results(df, dft_list):
    results = []
    for i, row in df.iterrows():
        factors = row.factors
        if all(factor in dft_list for factor in factors):
            results.append(row)
    return pd.DataFrame(results)

df_result_dft_only = filter_model_results(df_results, df_dft.columns.to_list())
df_result_dft_only.head()

Make prediction from DFT indicator properties, compare with the "true" DFT coefficient (calculated separately).

In [38]:
# best model of the 5 available DFT properties
dft_model_properties = ['c44_fcc',  
                        'unstable_stack_energy_fcc'
                       ]

df_dft['dft_pred_coeff'] = add_pred(df_dft, 
                                    pipe,
                                    dft_model_properties,
                                    label="dft_exact_coeff")

below is from dft_import. Need to figure out what to roll in. Without prediction

In [39]:
def get_boxplot(df_ip, 
                df_dft, 
                plot_errorbar = True,
                order_list = ["Ag","Al","Au","Cu","Ni","Pd","Pt"]):
    """plot boxplot of dft GB results
    """
    sns.set_theme(style="whitegrid")
    fig, ax = plt.subplots(figsize=(4,3))
    sns.boxplot(data = df_ip, 
                x="species", 
                y="coeff", 
                order=order_list, 
                color = "0.8", 
                linewidth=1.0,
                fliersize=5.0,
                whis=0,
                flierprops={"marker":"."},
                zorder=1)
    ax.set_ylabel("GB scaling coefficient")

    # add dft Xs
    ax.scatter(df_dft['species'],
               df_dft['dft_pred_coeff'], 
               marker='x', 
               s=100., 
               alpha=1.0, 
               color="r",
               label='\n'.join(wrap(r"$E_0$ regression prediction using DFT indicator properties",20)),
               zorder=3)

    df_dft_gb_plot = df_dft[['species','dft_exact_coeff']].drop_duplicates()
    df_dft_gb_plot = df_dft_gb_plot[df_dft_gb_plot['species'].isin(order_list)]
    ax.scatter(df_dft_gb_plot['species'],
               df_dft_gb_plot['dft_exact_coeff'], 
               marker='<', 
               s=50., 
               alpha=0.9, 
               color="r",
               label='\n'.join(wrap(r"$E_0$ fit directly to DFT GB results",20)),
               zorder=2)

    # add errorbars if desired
    if plot_errorbar == True:
        ax.errorbar(df_dft['species'],
                    df_dft['regr_coeff'], 
                    yerr = (df_dft['regr_coeff_lower'],
                            df_dft['regr_coeff_upper']), 
                            fmt='.', 
                            markersize=0.0001, 
                            alpha=0.5, 
                            color="r",
                            #label='\n'.join(wrap("Predicted strength using DFT indicator properties",20)),
                            elinewidth=2.0,
                            capsize = 4)

    fig.legend(bbox_to_anchor = (0.05,0.9,0.85,.15),#(0.,1.02,1.,.102),
                    loc='lower left',
                    mode="expand",
                    ncol = 2,
                    fontsize= 8)

    return 

In [ ]:
get_boxplot(df_data, df_dft, plot_errorbar=False)